In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers==4.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 61.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=99e7ad884f6dbef6fab6eab18932f3503a3389f0197938891daae9721c4e96b2
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
import pandas as pd
from transformers import RobertaTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaModel, AdamW
import numpy as np
import shutil
from tqdm import tqdm
from configparser import ConfigParser
import os
import json
import argparse
import shutil
import sys
import transformers
from nltk.tokenize import sent_tokenize
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.simplefilter("ignore")

In [ ]:
def load_dataframes():
    df = {}
    df["train"] = pd.read_pickle("data/train.pkl")
    df["valid"] = pd.read_pickle("data/valid.pkl")
    df["test"] = pd.read_pickle("data/test.pkl")
    return df


class ClaimData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['text']
        self.evidence = dataframe['evidence_sents']
        self.targets = dataframe['rating']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        evidence = str(self.evidence[index])
        evidence = " ".join(evidence.split())

        inputs = self.tokenizer(
            text,
            evidence,
            truncation="longest_first",
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    target_names = [0, 1, 2]
    output_dict = classification_report(labels, predictions, labels=range(3), target_names=target_names, output_dict=True)
    return {'accuracy': accuracy, 'macro f1-score': output_dict['macro avg']['f1-score']}




In [ ]:
rand_seed = 42
torch.manual_seed(rand_seed)
num_epochs = 3
output_dir = "output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

df = load_dataframes()

model_checkpoint = "output/checkpoint-5620"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

MAX_SEQ_LEN = 512
print("creating data....")
training_set = ClaimData(df['train'], tokenizer, MAX_SEQ_LEN)
validation_set = ClaimData(df['valid'], tokenizer, MAX_SEQ_LEN)
testing_set = ClaimData(df['test'], tokenizer, MAX_SEQ_LEN)
print("setting model....")
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

args = TrainingArguments(
    output_dir = output_dir,
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate=5e-6,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=num_epochs,
    weight_decay=0.001,
    load_best_model_at_end=True,
    metric_for_best_model='macro f1-score',
    seed=rand_seed
)

trainer = Trainer(
    model,
    args,
    train_dataset=training_set,
    eval_dataset=validation_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Didn't find file output/checkpoint-5620/added_tokens.json. We won't load it.
loading file output/checkpoint-5620/vocab.json
loading file output/checkpoint-5620/merges.txt
loading file output/checkpoint-5620/tokenizer.json
loading file None
loading file output/checkpoint-5620/special_tokens_map.json
loading file output/checkpoint-5620/tokenizer_config.json
loading configuration file output/checkpoint-5620/config.json
Model config RobertaConfig {
  "_name_or_path": "output/checkpoint-5620",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position

creating data....
setting model....


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at output/checkpoint-5620.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
predictions, labels, metrics = trainer.predict(testing_set)
metrics["predict_samples"] = len(testing_set)

trainer.log_metrics("predict", metrics)
trainer.save_metrics("predict", metrics)

predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(output_dir, "predictions.csv")

***** Running Prediction *****
  Num examples = 2357
  Batch size = 6


***** predict metrics *****
  predict_samples         =       2357
  test_accuracy           =     0.3975
  test_loss               =     1.2556
  test_macro f1-score     =     0.1896
  test_runtime            = 0:02:28.11
  test_samples_per_second =     15.913
  test_steps_per_second   =      2.653


In [ ]:
test_ids = df["test"]["claim_id"].values
import csv
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "rating"])
        for index, item in zip(test_ids, predictions):
            writer.writerow([index,item])
        writer.writerow([30380, 0])
        writer.writerow([31035, 0])
        writer.writerow([31420, 0])